In [2]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


0it [00:00, ?it/s]

9920512it [00:00, 13207377.01it/s]                            


Extracting MNIST/data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/data/MNIST/raw


32768it [00:00, 328760.43it/s]
0it [00:00, ?it/s]

Extracting MNIST/data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/data/MNIST/raw


1654784it [00:00, 3458314.20it/s]                            
8192it [00:00, 129917.26it/s]


Extracting MNIST/data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/data/MNIST/raw
Extracting MNIST/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/data/MNIST/raw
Processing...
Done!


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1005
Epoch [1/5], Step [200/600], Loss: 0.1260
Epoch [1/5], Step [300/600], Loss: 0.1201
Epoch [1/5], Step [400/600], Loss: 0.0933
Epoch [1/5], Step [500/600], Loss: 0.1302
Epoch [1/5], Step [600/600], Loss: 0.0355
Epoch [2/5], Step [100/600], Loss: 0.0449
Epoch [2/5], Step [200/600], Loss: 0.0478
Epoch [2/5], Step [300/600], Loss: 0.0219
Epoch [2/5], Step [400/600], Loss: 0.0165
Epoch [2/5], Step [500/600], Loss: 0.0555
Epoch [2/5], Step [600/600], Loss: 0.0708
Epoch [3/5], Step [100/600], Loss: 0.0697
Epoch [3/5], Step [200/600], Loss: 0.0495
Epoch [3/5], Step [300/600], Loss: 0.0092
Epoch [3/5], Step [400/600], Loss: 0.0266
Epoch [3/5], Step [500/600], Loss: 0.0050
Epoch [3/5], Step [600/600], Loss: 0.0538
Epoch [4/5], Step [100/600], Loss: 0.0911
Epoch [4/5], Step [200/600], Loss: 0.0232
Epoch [4/5], Step [300/600], Loss: 0.0537
Epoch [4/5], Step [400/600], Loss: 0.0082
Epoch [4/5], Step [500/600], Loss: 0.0363
Epoch [4/5], Step [600/600], Loss:

In [0]:
correction_dict = {}
correction_dict[0] = []
correction_dict[1] = []
correction_dict[2] = []
correction_dict[3] = []
correction_dict[4] = []
correction_dict[5] = []
correction_dict[5] = []
correction_dict[6] = []
correction_dict[7] = []
correction_dict[8] = []
correction_dict[9] = []

correct_test_dataset = []

In [8]:
# Test the model
model.eval()  # eval mode
with torch.no_grad():
    correct = 0
    total = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        for j in range(batch_size):
          if predicted[j] == labels[j]:
            correction_dict[labels[j].item()].append(idx*batch_size+j)
            correct_test_dataset.append(idx*batch_size+j)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 98.96 %


In [9]:
len(correct_test_dataset)

9896

In [0]:
# Save correctly predicted test datasets
path = F"/content/gdrive/My Drive/correct_test_dataset.txt" 
np.savetxt(path, correct_test_dataset, delimiter=",", fmt='%i')
path = F"/content/gdrive/My Drive/correction_dict.npy" 
np.save(path, correction_dict)


In [0]:
# Save the weights of the model's layers
path = F"/content/gdrive/My Drive/layer1_weights.npy" 
np.save(path, model.layer1[0].weight.data.cpu())
path = F"/content/gdrive/My Drive/layer2_weights.npy" 
np.save(path, model.layer2[0].weight.data.cpu())
path = F"/content/gdrive/My Drive/fc_weights.npy" 
np.save(path, model.fc.weight.data.cpu())
